In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [11]:
def user_function_emissions_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['routeLength'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model (GP)

Our surrogate model is our emulator. In this case, a gaussian process. I think that emukit handles categorical/discrete inputs using one-hot encodings.

#### Model Inputs

In [8]:
# gridSize = ContinuousParameter('gridSize', min_value=3, max_value=20)
# edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=1, max_value=25)
# maxSpeed = ContinuousParameter('maxSpeed', min_value=1, max_value=25)
# edgeLength = ContinuousParameter('edgeLength', min_value=30, max_value=200)
# numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1,2,3])
# accel = ContinuousParameter('accel', 1., 6.)

# parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numberOfLanes, accel])
import experimental_design.config as config
parameter_space = config.get_parameter_space()

#### Initialize Model / Emulator (GP)

In [12]:
#init_X, init_Y = pickle.load(open('experimental_design/init_points/25_init_points.pkl', 'rb'))

design = RandomDesign(parameter_space)  # initialize with random points
num_init_points = 20
init_X = design.get_samples(num_init_points)
init_Y = user_function_emissions_loss(init_X)
with open(f'experimental_design/init_points/emissions_{num_init_points}_init_points.pkl', "wb") as f:
      pickle.dump((init_X, init_Y), f)


[[ 8.          8.19230981 21.62866715 63.7582026   2.          4.29123225]
 [20.         14.63838354  8.28321899 58.8283849   2.          1.55048002]
 [ 6.          8.96980812 26.83845786 49.80998344  2.          1.69942663]
 [ 6.          9.7272656  22.24175249 54.11562883  2.          2.86534085]
 [13.         17.27631412 10.13217716 61.37356139  2.          2.97886077]
 [ 3.         11.34142697 26.45929545 41.1400627   1.          4.21806791]
 [ 8.          8.33280713  7.52313752 43.87167425  2.          3.99902165]
 [12.         13.8037289  21.37517605 47.99152096  3.          4.2437152 ]
 [16.         24.94729146  5.38565608 59.80612663  3.          4.2041301 ]
 [14.         24.75320592 20.34532765 53.94090547  3.          2.04589894]
 [13.         12.00935124 16.69112907 33.03326902  3.          4.74609975]
 [13.         11.20356256  7.35853681 47.47069743  1.          3.35720453]
 [20.          8.50180419 25.8100046  49.02781399  1.          2.38686677]
 [ 9.         21.69392584

In [13]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model = GPyModelWrapper(emulator, n_restarts=5)
emukit_model.optimize()
emulator

Optimization restart 1/5, f = 72.9465075322527
Optimization restart 2/5, f = 72.94650753105084
Optimization restart 3/5, f = 72.94650754203712
Optimization restart 4/5, f = 58.12430099637144
Optimization restart 5/5, f = 72.94650753105059


GP_regression.,value,constraints,priors
rbf.variance,165.87035625433424,+ve,
rbf.lengthscale,24.162414303923246,+ve,
Gaussian_noise.variance,3.760320799541241,+ve,


### Optimization

#### Acquisition Function

In [14]:
model_variance = ModelVariance(model=emukit_model)

#### Optimizer

In [15]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design

In [16]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
)

In [17]:
ed_loop.run_loop(user_function_emissions_loss, 200)

Optimization restart 1/5, f = 58.12430099263955
Optimization restart 2/5, f = 72.9465075625557
Optimization restart 3/5, f = 58.12430098631889
Optimization restart 4/5, f = 72.94650753134582
Optimization restart 5/5, f = 72.94650753154309
[[ 3. 25.  5. 30.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 61.01143022327771
Optimization restart 2/5, f = 77.10095098237356
Optimization restart 3/5, f = 77.1009509845871
Optimization restart 4/5, f = 61.01143022401521
Optimization restart 5/5, f = 77.10095098237416
[[20.  25.  27.  30.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 63.81779326694571
Optimization restart 2/5, f = 80.6286234864931
Optimization restart 3/5, f = 80.6286234864964
Optimization restart 4/5, f = 80.6286234865238
Optimization restart 5/5, f = 80.62862351853688
[[ 3. 2

 Retrying in 1 seconds
Optimization restart 1/5, f = 119.67245570755766
Optimization restart 2/5, f = 142.3909736869792
Optimization restart 3/5, f = 159.68704613764066
Optimization restart 4/5, f = 159.68704613803504
Optimization restart 5/5, f = 119.67245570750515
[[20.          8.          5.         53.33711168  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 121.65089723849815
Optimization restart 2/5, f = 162.99504012801808
Optimization restart 3/5, f = 121.6489897166216
Optimization restart 4/5, f = 162.99503984116978
Optimization restart 5/5, f = 162.9950401274983
[[ 3.  25.   5.  30.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 123.93810460456692
Optimization restart 2/5, f = 123.9372253656127
Optimization restart 3/5, f = 167.38207457921158
Optimization resta

 Retrying in 1 seconds
Optimization restart 1/5, f = 193.9866511229391
Optimization restart 2/5, f = 248.5603094239014
Optimization restart 3/5, f = 193.9866511200073
Optimization restart 4/5, f = 193.98665112064307
Optimization restart 5/5, f = 248.56030942366587
[[ 3. 25.  5. 70.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 196.31041634125486
Optimization restart 2/5, f = 252.154846233035
Optimization restart 3/5, f = 252.15484623786173
Optimization restart 4/5, f = 252.15484623314947
Optimization restart 5/5, f = 196.31037229805904
[[ 3.          8.         16.26682566 70.          3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 198.55863824530326
Optimization restart 2/5, f = 255.78050657498602
Optimization restart 3/5, f = 255.78050658741736
Optimization restart 4/5,

[[12.  8. 27. 30.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 239.4201925341252
Optimization restart 2/5, f = 322.88111599089933
Optimization restart 3/5, f = 322.8811168897225
Optimization restart 4/5, f = 322.8811166718867
Optimization restart 5/5, f = 322.8811168949445
[[11.  25.  27.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 243.65839102882921
Optimization restart 2/5, f = 243.6583910292992
Optimization restart 3/5, f = 326.41323661603224
Optimization restart 4/5, f = 243.65839102894822
Optimization restart 5/5, f = 245.6731871469047
[[20.         17.87592175  5.         30.          1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 246.4348957561469
Optimi

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


...

Optimization restart 3/5, f = 485.103039424602
Optimization restart 4/5, f = 758.0106327959894
Optimization restart 5/5, f = 758.0106328153892
[[14.          8.         21.72718718 70.          3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 486.559046537676
Optimization restart 2/5, f = 486.5589996415306
Optimization restart 3/5, f = 486.5589995847365
Optimization restart 4/5, f = 761.4985854641501
Optimization restart 5/5, f = 761.498585505326
[[20.        25.        14.0679592 49.2579813  1.         1.5      ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 487.96187235561086
Optimization restart 2/5, f = 765.0121366964543
Optimization restart 3/5, f = 487.9618394410706
Optimization restart 4/5, f = 765.0121366908633
Optimization restart 5/5, f = 765.0121366925115
[[11.         25

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = 497.1590117038829
[[ 3. 25. 27. 70.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 499.8117245455952
Optimization restart 2/5, f = 787.5504672703494
Optimization restart 3/5, f = 499.8115608017121
Optimization restart 4/5, f = 499.8115608036146
Optimization restart 5/5, f = 499.8115608064701
[[14.          8.          9.88023028 30.          3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 501.41043942016705
Optimization restart 2/5, f = 501.4102711361144
Optimization restart 3/5, f = 501.4102711362966
Optimization restart 4/5, f = 501.41027113934916
Optimization restart 5/5, f = 501.4102711361162
[[ 3.          8.         27.         61.51502613  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating inpu

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = 578.616998302919
[[20.         25.         27.         41.53110978  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 528.0178745653907
Optimization restart 2/5, f = 553.1217331200033
Optimization restart 3/5, f = 528.017872839654
Optimization restart 4/5, f = 843.0485879762427
Optimization restart 5/5, f = 843.0485879751914
[[ 3.         13.47101849 27.         42.09890618  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 530.4031392816971
Optimization restart 2/5, f = 847.7725685324967
Optimization restart 3/5, f = 530.4029370218544
Optimization restart 4/5, f = 530.4029370350148
Optimization restart 5/5, f = 554.9146969382056
[[20.          8.         10.61776721 36.09269448  1.          1.5       ]]

User function time loss calle

In [13]:
#emukit_model = pickle.load(open('experimental_design/models/5_init_50_loop_model_timeLoss.pkl', 'rb'))

### Evaluate

#### Test on 50 point test set

In [19]:
#test_X, test_Y = pickle.load(open('experimental_design/test_points/25_test_points.pkl', 'rb'))


design = RandomDesign(parameter_space)  # initialize with random points
num_test_points = 50
test_X = design.get_samples(num_test_points)
test_Y = user_function_emissions_loss(test_X)
with open(f'experimental_design/test_points/emissions_{num_test_points}_test_points.pkl', "wb") as f:
      pickle.dump((test_X, test_Y), f)


[[20.          9.99321136 12.45044307 42.12934287  2.          4.12683108]
 [13.         19.66213414 11.36585393 36.86879553  1.          3.64871866]
 [11.         11.02401122 24.8591953  58.42579369  3.          4.04996025]
 [17.         13.33723018 11.52853008 61.98380227  3.          1.64442037]
 [ 3.         19.29112263  9.6272481  36.35664966  3.          3.96095344]
 [11.         24.25724669 12.2373261  46.06147122  2.          2.78736566]
 [ 8.         12.79722749  5.38310225 39.79283957  3.          4.74401037]
 [ 5.         14.86435545 26.1432854  60.13598977  2.          4.68974886]
 [16.         13.8800419  16.59220905 67.07447216  3.          3.54750919]
 [ 9.         17.34579188 15.6874918  36.12910908  3.          3.14019047]
 [ 6.         17.39599601 19.9949168  53.18457944  1.          2.03241666]
 [17.         15.07124216 11.00448089 54.70776397  2.          3.99834739]
 [16.         22.44004348 12.88746058 53.31168146  2.          4.84668119]
 [15.         22.26854016

In [20]:
pred_Y, pred_Y_variance = emukit_model.predict(test_X)
mse = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Root mean squared error: ', mse)

Root mean squared error:  1.6538281955896075


In [21]:
pred_Y*100, test_Y*100

(array([[ 413.67747562],
        [ 860.08806507],
        [ 535.76748614],
        [ 304.5730834 ],
        [2349.49527112],
        [ 712.23111029],
        [ 723.07539435],
        [1461.98581722],
        [ 386.82506838],
        [1690.70905515],
        [1181.68405964],
        [ 376.03068498],
        [ 539.64511468],
        [ 609.33395686],
        [ 360.06935733],
        [ 409.18325812],
        [1076.08741259],
        [ 714.60820146],
        [1016.18206147],
        [1075.33032893],
        [1349.70192696],
        [ 529.55600074],
        [ 972.37287969],
        [ 766.40241248],
        [ 638.93388973],
        [2001.50272846],
        [1263.60714458],
        [ 552.23319347],
        [2739.64172404],
        [ 886.63726534],
        [ 742.12434554],
        [1055.30521449],
        [ 569.88283698],
        [ 456.50617907],
        [ 609.05694173],
        [1158.49704902],
        [ 421.15885576],
        [ 326.07637423],
        [ 278.43968794],
        [ 346.7807427 ],


### Save Model

In [22]:
num_init_points = 20
num_loop_iters = 200


with open(f'experimental_design/models/{num_init_points}_init_{num_loop_iters}_loop_model_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model, f)

## Baseline against a GP fitted to 200 random points

In [23]:
design = RandomDesign(parameter_space)  # initialize with random points
num_init_points = 200
init_X = design.get_samples(num_init_points)
init_Y = user_function_emissions_loss(init_X)
with open(f'experimental_design/init_points/emissions_{num_init_points}_init_points.pkl', "wb") as f:
      pickle.dump((init_X, init_Y), f)

[[ 7.         19.18103995  5.04867916 37.01630314  1.          4.89589923]
 [ 3.         14.41979111  8.25335159 66.63794093  1.          3.38033958]
 [20.          8.79942755 19.52561349 69.71401747  1.          4.01285395]
 ...
 [ 4.         16.09665991 11.32953786 31.53075428  2.          1.76838419]
 [10.         15.40921285 14.37447069 33.14876698  3.          2.28026889]
 [18.         14.05576178 24.71250904 31.86275543  3.          3.46285516]]

User function time loss called with 200 inputs to simulate

Evaluating input: 1 of 200

 Retrying in 1 seconds

Evaluating input: 2 of 200

 Retrying in 1 seconds

Evaluating input: 3 of 200

 Retrying in 1 seconds

Evaluating input: 4 of 200

 Retrying in 1 seconds

Evaluating input: 5 of 200

 Retrying in 1 seconds

Evaluating input: 6 of 200

 Retrying in 1 seconds

Evaluating input: 7 of 200

 Retrying in 1 seconds

Evaluating input: 8 of 200

 Retrying in 1 seconds

Evaluating input: 9 of 200

 Retrying in 1 seconds

Evaluating inpu

 Retrying in 1 seconds

Evaluating input: 146 of 200

 Retrying in 1 seconds

Evaluating input: 147 of 200

 Retrying in 1 seconds

Evaluating input: 148 of 200

 Retrying in 1 seconds

Evaluating input: 149 of 200

 Retrying in 1 seconds

Evaluating input: 150 of 200

 Retrying in 1 seconds

Evaluating input: 151 of 200

 Retrying in 1 seconds

Evaluating input: 152 of 200

 Retrying in 1 seconds

Evaluating input: 153 of 200

 Retrying in 1 seconds

Evaluating input: 154 of 200

 Retrying in 1 seconds

Evaluating input: 155 of 200

 Retrying in 1 seconds

Evaluating input: 156 of 200

 Retrying in 1 seconds

Evaluating input: 157 of 200

 Retrying in 1 seconds

Evaluating input: 158 of 200

 Retrying in 1 seconds

Evaluating input: 159 of 200

 Retrying in 1 seconds

Evaluating input: 160 of 200

 Retrying in 1 seconds

Evaluating input: 161 of 200

 Retrying in 1 seconds

Evaluating input: 162 of 200

 Retrying in 1 seconds

Evaluating input: 163 of 200

 Retrying in 1 seconds

Eval

In [28]:
emulator_rnd = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=200)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/200, f = 479.9159621278644
Optimization restart 2/200, f = 407.882682222363
Optimization restart 3/200, f = 727.6875970035791
Optimization restart 4/200, f = 407.88268223014654
Optimization restart 5/200, f = 407.88269295821624
Optimization restart 6/200, f = 479.91596213583864
Optimization restart 7/200, f = 407.8826822153346
Optimization restart 8/200, f = 407.8826822155627
Optimization restart 9/200, f = 727.6875970036642
Optimization restart 10/200, f = 727.6875968823547
Optimization restart 11/200, f = 727.687596946859
Optimization restart 12/200, f = 407.88269208809345
Optimization restart 13/200, f = 407.8826822152424
Optimization restart 14/200, f = 407.88268228266395
Optimization restart 15/200, f = 407.8826822151677
Optimization restart 16/200, f = 407.8826822158792
Optimization restart 17/200, f = 407.88268222446806
Optimization restart 18/200, f = 407.88268221650503
Optimization restart 19/200, f = 407.8826824110399
Optimization restart 20/200, f = 40

Optimization restart 160/200, f = 407.8826822182858
Optimization restart 161/200, f = 407.88268221515557
Optimization restart 162/200, f = 727.6875970041317
Optimization restart 163/200, f = 727.6875970037713
Optimization restart 164/200, f = 407.88268222001545
Optimization restart 165/200, f = 407.88268324070987
Optimization restart 166/200, f = 407.8826822166642
Optimization restart 167/200, f = 407.8826822152187
Optimization restart 168/200, f = 407.8826822151499
Optimization restart 169/200, f = 727.6875970037563
Optimization restart 170/200, f = 407.8826822154579
Optimization restart 171/200, f = 727.6875970035787
Optimization restart 172/200, f = 407.88268221518456
Optimization restart 173/200, f = 407.88268221521855
Optimization restart 174/200, f = 727.6875970035794
Optimization restart 175/200, f = 727.6875970035785
Optimization restart 176/200, f = 407.8826822504933
Optimization restart 177/200, f = 407.8826822151407
Optimization restart 178/200, f = 407.8826822152115
Optimiz

GP_regression.,value,constraints,priors
rbf.variance,593.9947813574224,+ve,
rbf.lengthscale,26.576153825204457,+ve,
Gaussian_noise.variance,1.5630862701588542,+ve,


In [29]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Root mean squared error: ', mse)

Root mean squared error:  2.1996383697836333


In [30]:
1- (1.6538281955896075 / 2.1996383697836333)

# Experimental design is ~25% better than random.

0.24813632172079003